In [ ]:
import platform
import subprocess
from ipaddress import ip_address

# BLOCKING:
def ping(host, timeout=5.0):
    """
    Returns True if host (str) responds to a ping request.
    Remember that a host may not respond to a ping (ICMP) request even if the host name is valid.
    """
    # check for validity; ValueError if invalid
    host = ip_address(host)
    
    # Option for the number of packets as a function of
    param = '-n' if platform.system().lower()=='windows' else '-c'

    # Building the command. Ex: "ping -c 1 google.com"
    command = ['ping', param, '1', str(host)]

    try:
        val = subprocess.call(command, timeout=timeout)
        return val == 0
    except subprocess.TimeoutExpired:
        return False

In [ ]:
ping("127.0.0.1", timeout=.2)

In [ ]:
ping("127.0.0.0", timeout=.2)

In [36]:
from subprocess import Popen, PIPE
from ipaddress import ip_address
import platform
import time

# ASYNCHRONOUS AND PARALLEL
class Class:
    
    def ping_ips(self, hosts):
        
        # Check if running, return if so
        
        self.ping_procs = []
        self.ping_values = []
        
        for host in hosts:

            # check for validity; ValueError if invalid
            host = ip_address(host)

            # Option for the number of packets as a function of
            param = '-n' if platform.system().lower()=='windows' else '-c'

            # Building the command. Ex: "ping -c 1 google.com"
            command = ['ping', param, '1', str(host)]

            self.ping_procs += [Popen(command,stdout=PIPE)]
        
        self.ping_values = [None for p in self.ping_procs]
        
    def check_pings(self):
        
        for i, p in enumerate(self.ping_procs):
            
            if p.poll() is not None:
                # Done running, get output
                try:
                    out,err = p.communicate()
                    self.ping_values[i] = str(out,'utf-8')
                    p.kill()
                except ValueError:
                    # p has already been killed
                    pass
                
        return self.ping_values

In [37]:
c = Class()

In [47]:
c.ping_ips(['127.0.0.1', '9.0.0.2', '11.0.0.1', '140.82.112.4'])

In [86]:
c.ping_procs

In [85]:
for p in c.ping_procs:
    p.terminate()

In [87]:
pings = c.check_pings()

for p in pings:
    print('>>>', p)

>>> PING 127.0.0.1 (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.042 ms

--- 127.0.0.1 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.042/0.042/0.042/0.000 ms

>>> PING 9.0.0.2 (9.0.0.2): 56 data bytes

--- 9.0.0.2 ping statistics ---
1 packets transmitted, 0 packets received, 100.0% packet loss

>>> PING 11.0.0.1 (11.0.0.1): 56 data bytes

--- 11.0.0.1 ping statistics ---
1 packets transmitted, 0 packets received, 100.0% packet loss

>>> PING 140.82.112.4 (140.82.112.4): 56 data bytes
64 bytes from 140.82.112.4: icmp_seq=0 ttl=37 time=85.321 ms

--- 140.82.112.4 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 85.321/85.321/85.321/0.000 ms



In [79]:
def parse_ping(ping):
    """ macOS ping parsing """
    
    value = "-- ms"
    
    if ping is None:
        return value
    
    blob = ping.split('\n')[1]    

    if blob == '':
        return value
    
    if ' ' in blob:
        blob = blob.split(' ')

        if len(blob) > 3:
            value = blob[-2].split('=')[1] + " ms"
            
    return value

In [80]:
parse_ping(pings[0])

'0.042 ms'

In [81]:
parse_ping(pings[1])

'-- ms'

In [82]:
parse_ping(pings[3])

'85.321 ms'

In [122]:
from subprocess import Popen, PIPE

# ASYNCHRONOUS AND PARALLEL
class ParallelApplication:
    
    def run_commands(self, commands, errors=False):
        
        # Check if running, return if so
        
        self.procs = []
        self.returns = []
        
        for cmd in commands:
            if errors:
                stderr = PIPE
            else:
                stderr = None
                
            self.procs += [Popen(cmd, stdout=PIPE, stderr=stderr)]
        
        self.returns = [None for p in self.procs]
        
    def check_returns(self):
        
        for i, p in enumerate(self.procs):
            
            if p.poll() is not None:
                # Done running, get output
                try:
                    out,err = p.communicate()
                    self.returns[i] = str(out,'utf-8')
                    p.kill()
                    
                except ValueError:
                    # p has already been killed
                    pass
                
        return self.returns

In [130]:
hosts = ['127.0.0.1', '9.0.0.1', "10.0.0.134"]

commands = []

for host in hosts:

    # check for validity; ValueError if invalid
    host = ip_address(host)

    # Option for the number of packets as a function of
    param = '-n' if platform.system().lower()=='windows' else '-c'

    # Building the command. Ex: "ping -c 1 google.com"
    command = ['ping', param, '1', str(host)]
    
    commands += [command]

In [131]:
p = ParallelApplication()
p.run_commands(commands)

In [136]:
for r in p.check_returns():
    print(r)

PING 127.0.0.1 (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.060 ms

--- 127.0.0.1 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.060/0.060/0.060/0.000 ms

PING 9.0.0.2 (9.0.0.2): 56 data bytes

--- 9.0.0.2 ping statistics ---
1 packets transmitted, 0 packets received, 100.0% packet loss

PING 10.0.0.134 (10.0.0.134): 56 data bytes
64 bytes from 10.0.0.134: icmp_seq=0 ttl=128 time=4.363 ms

--- 10.0.0.134 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 4.363/4.363/4.363/0.000 ms



In [140]:
p.check_returns()[0].split('\n')

['PING 127.0.0.1 (127.0.0.1): 56 data bytes',
 '64 bytes from 127.0.0.1: icmp_seq=0 ttl=64 time=0.060 ms',
 '',
 '--- 127.0.0.1 ping statistics ---',
 '1 packets transmitted, 1 packets received, 0.0% packet loss',
 'round-trip min/avg/max/stddev = 0.060/0.060/0.060/0.000 ms',
 '']